In [3]:
import torch
from diffusers import StableDiffusion3Pipeline # type: ignore
# 저장된 모델 경로
save_path = "/home/oss_1/MinsuKim/stable-diffusion-3.5-model"
# 모델 불러오기
pipe = StableDiffusion3Pipeline.from_pretrained(save_path, torch_dtype=torch.bfloat16)
pipe = pipe.to("cuda")

In [4]:
import os

# GPU 메모리 사용 설정
torch.cuda.set_per_process_memory_fraction(0.7, device=torch.cuda.current_device())
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [5]:


import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, image_root, label_root, transform=None):
        self.image_root = image_root
        self.label_root = label_root
        self.transform = transform
        self.image_paths = []
        self.label_paths = []

        # 이미지와 레이블 파일 경로를 일치시키는 작업
        for image_folder in os.listdir(image_root):
            if image_folder.endswith('.zip') or image_folder == '.ipynb_checkpoints':
                continue
            image_folder_path = os.path.join(image_root, image_folder)
            label_folder_path = os.path.join(label_root, f"TL_TEXT_{image_folder.split('_')[1]}_{image_folder.split('_')[2]}")

            for image_file in os.listdir(image_folder_path):
                label_file = image_file.split(".")[0] + ".txt"
                self.image_paths.append(os.path.join(image_folder_path, image_file))
                self.label_paths.append(os.path.join(label_folder_path, label_file))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_path = self.label_paths[idx]

        # 이미지 로드 및 전처리
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # 레이블 로드
        with open(label_path, "r", encoding="utf-8") as file:
            label = file.read().strip()  # 레이블 파일 내용 읽기

        return image, label

# 이미지 전처리 설정 (1024x1024 크기로 조정, 정규화)
transform = transforms.Compose([
    transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 데이터셋 및 DataLoader 생성
train_image_root = "/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터"
train_label_root = "/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/02.라벨링데이터"

train_dataset = CustomDataset(train_image_root, train_label_root, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers= 8, pin_memory=True)

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [12]:
import torch
from transformers import AdamW
from accelerate import Accelerator
from tqdm import tqdm  # tqdm 라이브러리 임포트
from datetime import datetime

# 텍스트 인코더만 가져오기
text_encoder = pipe.text_encoder

# 옵티마이저 및 Accelerator 설정
optimizer = AdamW(text_encoder.parameters(), lr=1e-5)
accelerator = Accelerator()
text_encoder, optimizer = accelerator.prepare(text_encoder, optimizer)

# 학습 루프
num_epochs = 3  # 학습 에포크 수
for epoch in range(num_epochs):
    for images, captions in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch"):  # tqdm 적용
        images = images.to(accelerator.device)

        # 텍스트 인코딩
        text_inputs = pipe.tokenizer(captions, return_tensors="pt", padding=True, truncation=True)
        text_embeddings = text_encoder(text_inputs.input_ids.to(accelerator.device))[0]

        # 임시 손실 계산 (예시로 MSE 사용, 필요에 맞게 수정 가능)
        noise = torch.randn_like(text_embeddings).to(accelerator.device)
        loss = torch.nn.functional.mse_loss(text_embeddings, noise)

        # 역전파 및 옵티마이저 업데이트
        accelerator.backward(loss)
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

print("Fine-tuning Complete!")
# 학습 종료 후 모델 저장
save_path = "/home/oss_1/MinsuKim/stable-diffusion-3.5-model_" + datetime.now().strftime("%Y%m%d_%H%M%S")
pipe.save_pretrained(save_path)  # 모델 저장 경로 지정


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


KeyboardInterrupt: 

: 

In [13]:
import torch
import torch.nn.functional as F
from transformers import CLIPTextModel, CLIPTokenizer
from tqdm import tqdm  # tqdm 라이브러리 임포트

# CLIP 모델을 사용하여 텍스트를 임베딩합니다.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch16")
text_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch16").to(device)

# 학습 루프
num_epochs = 5
model.train()

for epoch in range(num_epochs):
    epoch_loss = 0  # 에포크 손실 초기화
    # tqdm을 사용하여 진행 바 추가
    for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch"):
        images = images.to(device)
        print(images)
        print(labels)
        # 텍스트 레이블을 토큰화하고 임베딩합니다.
        text_inputs = tokenizer(labels, padding=True, return_tensors="pt").to(device)
        text_embeddings = text_model(**text_inputs).last_hidden_state  # 텍스트 임베딩

        optimizer.zero_grad()

        # 랜덤 노이즈 생성
        noise = torch.randn(images.shape).to(device)
        timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (images.shape[0],)).long().to(device)

        # 노이즈 추가
        noisy_images = noise_scheduler.add_noise(images, noise, timesteps)

        # 모델 예측 및 손실 계산
        noise_pred = model(noisy_images, timesteps, text_embeddings).sample  # 텍스트 임베딩을 모델에 전달
        print(f"Predicted noise shape: {noise_pred.shape}")
        print(f"Actual noise shape: {noise.shape}")

        # 손실 계산 (MSE 손실)
        if noise_pred.shape != noise.shape:
            # 크기를 맞추기 위한 조정 (예: 슬라이싱 또는 패딩)
            noise_pred = noise_pred[:, :noise.shape[1]]  # 슬라이싱 예시
        # 손실 계산 (MSE 손실)
        loss = F.mse_loss(noise_pred, noise)

        # 역전파 및 가중치 업데이트
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()  # 에포크 손실 누적

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_dataloader):.4f}")

print("Training Finished!")

Epoch 1/5:   0%|                                   | 0/29187 [00:00<?, ?batch/s]

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
        

RuntimeError: The size of tensor a (336) must match the size of tensor b (77) at non-singleton dimension 1